# Using Joins effectively in Druid 
<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->

Apache Druid can do joins at query time through both its native engine (scatter/gather) and through the Multi-Stage Query engine.
Note that the best performance will always be without using any joins, so the overall recommendation is to join the data at ingestion when loading in batch, or upstream of ingestion for streaming scenarios. But pre-joining the data isn't always an option, so you'll need to learn how to use Druid to process joins as efficiently as possible for those cases.

Through this notebook you will run tests on the performance of different approaches to joining data, seeing how different JOIN operations are processed and how to write SQL that will take advantage of how it works in each case.
The notebook also covers alternatives to joins using UNION ALL strategies or application side code. 


## Prerequisites

This tutorial works with Druid 27.0.0 or later.

#### Run with Docker

Launch this tutorial and all prerequisites using the `druid-jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see the [learn-druid project page](https://github.com/implydata/learn-druid).
   

## Initialization

The following cells set up the notebook and learning environment ready for use.

### Set up and connect to the learning environment

Run the next cell to set up the Druid Python client's connection to Apache Druid and a client to interact with the Data Generator.

If successful, the Druid version number will be shown in the output.

In [ ]:
import druidapi
import os
import json
import pandas
import time
from datetime import datetime, timedelta
import requests

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"
    
print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)

display = druid.display
sql_client = druid.sql
status_client = druid.status

print(f"Version:{status_client.version}")

# client for Data Generator API
datagen = druidapi.rest.DruidRestClient("http://datagen:9999")

The following helper function is used throughout the examples in this notebook to provide a timeline of execution that can then be compared to the Docker container stats history to examine CPU, Memory, I/O and Network activity for each operation.

In [ ]:
#function to record progress timestamps so they can be compared to resource utilization metrics
def print_timestamp( msg = '' ):
    t = datetime.now()
    print(f"{t} <<<<<<<<<<<<< {msg}")
    return t

## Generate Data
Run the following cells which will:
- Data for clicks using the data generators `clickstream/clickstream.json` configuration.
- Data for users with the `clickstream/users_init.json` configuration.
These two data generator configs are compatible because they share a domain for the user_id field values. In other words, they are joinable.
  

In [ ]:
def wait_for_datagen( job_name:str):
    import time
    from IPython.display import clear_output
    # wait for the messages to be fully published 
    done = False
    while not done:
        result = datagen.get_json(f"/status/{job_name}",'')
        clear_output(wait=True)
        print(json.dumps(result, indent=2))
        if result["status"] == 'COMPLETE':
            done = True
        else:
            time.sleep(1)

In [ ]:
print_timestamp()

# simulate clicks for last 2 days
gen_hours=48
gen_now = datetime.now() - timedelta(hours=gen_hours)
gen_start_time = gen_now.strftime("%Y-%m-%d %H:%M:%S")

headers = {
  'Content-Type': 'application/json'
}

datagen_request = {
    "name": "clicks",
    "target": { "type": "file", "path":"clicks.json"  },
    "config_file": "clickstream/clickstream.json", 
    "time": f"{gen_hours}h",
    "concurrency":200,
    "time_type":gen_start_time
}
datagen.post("/start", json.dumps(datagen_request), headers=headers)

In [ ]:
wait_for_datagen( "clicks")  # this takes about 2 minutes
print_timestamp()

In [ ]:
print_timestamp()
datagen_request = {
    "name": "users",
    "target": { "type": "file", "path":"users.json"  },
    "config_file": "clickstream/users_init.json", 
    "concurrency":4000,
    "total_events":4000 
}
datagen.post("/start", json.dumps(datagen_request), headers=headers)

In [ ]:
wait_for_datagen( "users")
print_timestamp()

The data generation `target`s above use files "clicks.json" and "users.json" respectively as output files. These files are generated locally on the data generation server and can be accessed through HTTP at: 
- http://localhost:9999/file/clicks.json
- http://localhost:9999/file/users.json 

You will use those URLs below in the ingestion SQL's EXTERN table functions to load this data.

### Load `clicks` and `users` Into Separate Tables

Load separate tables for `clicks` and `users` to demonstrate join funtionality.

Instead of manually figuring out the EXTERN table syntax needed to build the ingestion SQL statement, you can generate the SQL statements from the data file by using the Druid Console and following these steps:

1. Click on `Load` and select `Batch-SQL`
   <img src="assets/load-data-batch.png" alt="drawing" style="width:400px;"/>
3. Select an HTTP source and type in the URL to one of the generated file, the data generator service makes them avaialable at:
   - `http://datagen:9999/file/users.json`
   - `http://datagen:9999/file/clicks.json`
4. Click on `Connect Data` and Druid Console will parse and present a few rows of data from the file.

   ![](assets/load-get-sql.png)
5. Click on `Skip the wizard and continue with custom SQL`, the SQL is displayed in a new Query view tab ready to edit or run.

### Execute the Batch Ingestions
This cell uses the files you generated above to load the two tables:

In [ ]:
# ingest users
sql='''
REPLACE INTO "users" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["http://datagen:9999/file/users.json"]}',
    '{"type":"json"}'
  )
) EXTEND ("time" VARCHAR, "user_id" VARCHAR, "first_name" VARCHAR, "last_name" VARCHAR, "dob" VARCHAR, "address_lat" VARCHAR, "address_long" VARCHAR, "marital_status" VARCHAR, "income" VARCHAR, "signup_ts" VARCHAR))
SELECT
  TIMESTAMP'1970-01-01 00:00:00' AS "__time",
  "user_id",
  "first_name",
  "last_name",
  "dob",
  "address_lat",
  "address_long",
  "marital_status",
  "income",
  "signup_ts"
FROM "ext"
PARTITIONED BY ALL
'''
display.run_task(sql)
print_timestamp()

# ingest clicks
sql='''
REPLACE INTO "clicks" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["http://datagen:9999/file/clicks.json"]}',
    '{"type":"json"}'
  )
) EXTEND ("time" VARCHAR, "user_id" VARCHAR, "event_type" VARCHAR, "client_ip" VARCHAR, "client_device" VARCHAR, "client_lang" VARCHAR, "client_country" VARCHAR, "referrer" VARCHAR, "keyword" VARCHAR, "product" VARCHAR))
SELECT
  TIME_PARSE("time") AS "__time",
  "user_id",
  "event_type",
  "client_ip",
  "client_device",
  "client_lang",
  "client_country",
  "referrer",
  "keyword",
  "product"
FROM "ext"
PARTITIONED BY DAY
'''
display.run_task(sql)
print_timestamp()

# make sure both tables are available before moving on
sql_client.wait_until_ready('users')
sql_client.wait_until_ready('clicks')
print_timestamp()

### Pre-join into Denormalized Table
In many cases it is best to pre-join the data. Druid is particularly suited for analytic applications that require subsecond response times in order to drive an interactive user experience with high concurrency. In a parallel system, avoiding query time joins is one of the tools that is useful to achieve high query performance at high concurrencies by avoiding the movement of data during queries.

The next cell ingests the two datasets into a single pre-joined table:

In [ ]:
# ingest clicks_enhanced including user data
sql='''
REPLACE INTO "clicks_enhanced" OVERWRITE ALL
WITH 
"users_ext" AS 
(
  SELECT *
  FROM TABLE(
    EXTERN(
      '{"type":"http","uris":["http://datagen:9999/file/users.json"]}',
      '{"type":"json"}'
    )
  ) EXTEND ("time" VARCHAR, "user_id" VARCHAR, "first_name" VARCHAR, "last_name" VARCHAR, "dob" VARCHAR, "address_lat" VARCHAR, "address_long" VARCHAR, "marital_status" VARCHAR, "income" VARCHAR, "signup_ts" VARCHAR)
),
"clicks_ext" AS 
(
  SELECT *
  FROM TABLE(
    EXTERN(
      '{"type":"http","uris":["http://datagen:9999/file/clicks.json"]}',
      '{"type":"json"}'
    )
  ) EXTEND ("time" VARCHAR, "user_id" VARCHAR, "event_type" VARCHAR, "client_ip" VARCHAR, "client_device" VARCHAR, "client_lang" VARCHAR, "client_country" VARCHAR, "referrer" VARCHAR, "keyword" VARCHAR, "product" VARCHAR)
)
SELECT
  TIME_PARSE(c."time") AS "__time",
  c."user_id",
  c."event_type",
  c."client_ip",
  c."client_device",
  c."client_lang",
  c."client_country",
  c."referrer",
  c."keyword",
  c."product",
  u."first_name",
  u."last_name",
  u."dob",
  TIMESTAMPDIFF(YEAR, TIME_PARSE(u."dob"), CURRENT_TIMESTAMP) as age,
  ROUND( TIMESTAMPDIFF(YEAR, TIME_PARSE(u."dob"), CURRENT_TIMESTAMP), -1) as age_group,
  u."address_lat",
  u."address_long",
  u."marital_status",
  u."income",
  u."signup_ts"
FROM "clicks_ext" c LEFT OUTER JOIN "users_ext" u ON c."user_id"=u."user_id"
PARTITIONED BY ALL
'''
display.run_task(sql)
print_timestamp()

### Load Users as a Lookup
Lookups are in-memory key/value tables that are pre-broadcast to all brokers, peons and historicals that belong to the lookup tier. They are faster to join to other tables because they are already in memory where they are needed.
The following cells prepare and load the user_age_group lookup to demonstrate the benefits of this join strategy further below in this notebook.


In [ ]:
# Define helper functions to load lookup
def postLookup(definition):
    x = requests.post(druid_host + '/druid/coordinator/v1/lookups/config', json=definition)

    if "error" in x.text:
        raise Exception('Not able to complete the request. \n\n'+x.text)
    else:
        print('Successfully submitted the lookup request.')

def waitForLookup(tier, name, ticsMax):

    # The default time period between checks of lookup definition changes (druid.manager.lookups.period)
    # is two minutes. The notebook environment reduces this for learning purposes.
    # 
    # https://druid.apache.org/docs/27.0.0/configuration/#lookups-dynamic-configuration

    tics = 0
    ticsWait = 1    
    ticsMax = min(ticsMax,360)
    ticsSpinner = "/-\|"
    
    apicall = druid_host + '/druid/coordinator/v1/lookups/status/'+tier+'/'+name+'?detailed=true'

    x = requests.get(apicall)

    while (x.text != '{"loaded":true,"pendingNodes":[]}' and tics < ticsMax):
        print(x.text + ' ' + ticsSpinner[tics%len(ticsSpinner)] + ' [' + str(ticsMax-tics) + ']   ', end='\r')
        time.sleep(ticsWait)
        tics += 1
        x = requests.get(apicall) 

    if (tics == ticsMax):
        raise Exception('\nTimeout waiting for Druid to load the ' + name + ' lookup to ' + tier + 'tier. Run the cell again.')
    else:
        print('\nSuccess. ' + name + ' lookup in ' + tier + ' tier is fully available.')
        
# initialize lookup environment        
empty_post = {}
postLookup(empty_post)

In [ ]:
# define user_age_group lookup by building it from a query of users
sql='''
SELECT user_id, ROUND( TIMESTAMPDIFF(YEAR, TIME_PARSE("dob"), CURRENT_TIMESTAMP),-1) as age_group
FROM "users"
'''
results = sql_client.sql(sql)
map = dict( zip([ r['user_id'] for r in results ], [ r['age_group'] for r in results ]))

lookup_tier = "__default"
lookup_name = "user_age_group"
lookup_definition_version = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
lookup_definition = {
    lookup_tier: {
        lookup_name: {
            "version": lookup_definition_version,  
            "lookupExtractorFactory": {
                "type": "map",
                    "map": map
            }
        }
    }
}
postLookup(lookup_definition)
waitForLookup(lookup_tier, lookup_name, 30)
print_timestamp()

## Prep Query Testing and Measurement Function
We measure each method of joining the data by running the query multiple times and avoiding cacheing of the results in order to get a avg, min and max runtimes of each query. We'll use the following function to achieve this:

In [ ]:
from statistics import mean 

def measure_query( sql: str, iterations: int ):
    req = sql_client.sql_request(sql)
    req.add_context("populateCache", "false")  # run without cacheing results to get a real sense of performance
    req.add_context("useCache", "false")  # do not use cached results
    stats = []
    while (iterations>0):
      start = datetime.now()
      sql_client.sql(req)
      end = datetime.now()
      stats.append( (end - start).total_seconds() * 10**3 ) # add run time in milliseconds
      iterations -=1
    return f"Results = avg:{mean(stats)} ms   min:{min(stats)} ms  max:{max(stats)} ms"

## Synchronous Queries on Scatter/Gather Engine
Druid's scatter/gather query processing is designed for speed.
### The Scatter/Gather Engine
At a high level, Druid queries run as follows:

<img src="assets/scatter-gather.png" alt="drawing" style="width:600px;"/>

1. Broker receives query
2. Broker plans query and determines which segments are relevant using cached segment metadata
3. Broker forwards query to data servers, telling each server what segments to interrogate
4. Data servers process segment files
5. Data servers send results back to Broker
6. Broker performs final processing and returns results

Note that the data servers do not directly communicate with each other.

### Joins with a Scatter/Gather Engine
When processing a join, the engine uses multiple scatter/gather steps. It broadcasts the results from the first scatter/gather in a second scatter/gather. The first table in the FROM clause is processed last which means its data does not need to move in order to process the join. This means that you should put the largest table at the front of the joins.
![](assets/join-scatter-gather.png)
1. The Broker receives the query request and creates a plan. The first datasource in the from clause becomes the driving table, meaning that it will process the join on this table in parallel across all Data Servers involved.
3. The Broker reads `users` from all data servers and merges them into a single dataset.
4. Merged `users` are broadcast to all data servers involved by adding them as an inline data source to the driving table request that is submitted to Data Servers.
5. Data Servers join the `clicks` data they hold with the inlined `users` which have all users.
6. To finish up this query, each Data Server processes its partial aggregation.
7. Finally, the Broker does final aggregation and returns result to the user.

Try it out:

In [ ]:
sql='''
SELECT TIME_FLOOR(c.__time, 'P1D') as _date, 
  round( TIMESTAMPDIFF(YEAR, TIME_PARSE(u.dob), CURRENT_TIMESTAMP),-1) as age_group, 
  count(distinct c.client_ip) as distint_ips
FROM clicks c 
     INNER JOIN users u
        ON c.user_id=u.user_id
WHERE c.__time > CURRENT_TIMESTAMP  - INTERVAL '1' DAY
GROUP BY 1, round( TIMESTAMPDIFF(YEAR, TIME_PARSE(u.dob), CURRENT_TIMESTAMP),-1) 
ORDER BY 3 DESC
LIMIT 10
'''
display.sql(sql)
print_timestamp()

Run it 20 times...

In [ ]:
join_results=[]
result = measure_query(sql, 20)
# saving all results for comparison 
join_results.append({"query_type":"simple_join","results":result})
print(result)
print_timestamp()

### Order of the Joins is Important

![](assets/fail-scatter-gather.png)

If you change the order of the joins, it will fail because the subquery will return more than `maxSubQueryRows` which defaults to 100000. 

In [ ]:
sql='''
SELECT TIME_FLOOR(c.__time, 'P1D') as _date, 
  round( TIMESTAMPDIFF(YEAR, TIME_PARSE(u.dob), CURRENT_TIMESTAMP),-1) as age_group, 
  count(distinct c.client_ip) as distint_ips
FROM users u
     INNER JOIN clicks c
        ON c.user_id=u.user_id
WHERE c.__time > CURRENT_TIMESTAMP  - INTERVAL '1' DAY
GROUP BY 1, round( TIMESTAMPDIFF(YEAR, TIME_PARSE(u.dob), CURRENT_TIMESTAMP),-1) 
ORDER BY 3 DESC
LIMIT 10
'''
try:
  display.sql(sql)
except Exception as ex:
  print(f"ERROR: {ex}")

### Nested Subqueries
See the query in the next cell, Druid will execute the subquery first as set of scatter/gather steps and then process the rest of the joins in subsequent scatter/gather steps. The results of the subquery still needs to be within the `maxSubQueryRows` limit, but running it this way will allow all processing of the larger `clicks` data with the parallelism available on the Data Servers: 
![](assets/joins-subquery.png)

The subquery calculates the total number of clicks per age group by day for for that last day. The outer query then calculates this by user and calculates each user's contribution percentage of the overall activity. The result shows the top 10 users by contribution percentage.

In [ ]:
sql='''
SELECT TIME_FLOOR(c.__time, 'P1D') as _date, c.user_id,
  round( TIMESTAMPDIFF(YEAR, TIME_PARSE(u.dob), CURRENT_TIMESTAMP),-1) as age_group, 
  count(1) as user_clicks,
  count(1)*100.0/agg.clicks as age_group_contrib_pct
FROM clicks c
     INNER JOIN users u ON c.user_id = u.user_id
     INNER JOIN
     (
         SELECT TIME_FLOOR(cc.__time, 'P1D') as _date,
                ROUND( TIMESTAMPDIFF(YEAR, TIME_PARSE(uu.dob), CURRENT_TIMESTAMP),-1) as age_group,
                COUNT(1) as clicks
           FROM clicks cc
           INNER JOIN users uu ON cc.user_id=uu.user_id
           WHERE cc.__time > CURRENT_TIMESTAMP  - INTERVAL '1' DAY
           GROUP BY 1, ROUND( TIMESTAMPDIFF(YEAR, TIME_PARSE(uu.dob), CURRENT_TIMESTAMP),-1)
     ) agg
        ON agg._date = TIME_FLOOR(c.__time, 'P1D') AND agg.age_group=round( TIMESTAMPDIFF(YEAR, TIME_PARSE(u.dob), CURRENT_TIMESTAMP),-1) 
WHERE c.__time > CURRENT_TIMESTAMP  - INTERVAL '1' DAY
GROUP BY 1, 2, round( TIMESTAMPDIFF(YEAR, TIME_PARSE(u.dob), CURRENT_TIMESTAMP),-1), agg.clicks 
ORDER BY 1, 5 DESC
LIMIT 10
'''
display.sql(sql)
print_timestamp()

In [ ]:
result = measure_query(sql, 20)
# saving all results for comparison 
join_results.append({"query_type":"subquery_join","results":result})
print_timestamp()
print(result)

### Join using a Lookup
Lookups in Druid are broadcast to all data servers at the time of creation and can be refreshed [continuously from a kafka stream](https://druid.apache.org/docs/latest/development/extensions-core/kafka-extraction-namespace) or refreshed periodically. The execution of the query can now be done in a single scatter/gather step:
![](assets/joins-lookup.png)

In [ ]:
sql='''
SELECT TIME_FLOOR(c.__time, 'P1D') as _date, 
  u.v as age_group, 
  count(distinct c.client_ip) as distint_ips
FROM clicks c 
     INNER JOIN lookup.user_age_group u
        ON c.user_id=u.k
WHERE c.__time > CURRENT_TIMESTAMP  - INTERVAL '1' DAY
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 10
'''

result = measure_query(sql, 20)
# saving all results for comparison 
join_results.append({"query_type":"simple_join_lookup","results":result})
print_timestamp()
print(result)

... and the second query using the `lookup`:

In [ ]:
sql = '''
SELECT TIME_FLOOR(c.__time, 'P1D') as _date, c.user_id,
  u.v as age_group, 
  count(1) as user_clicks,
  count(1)*100.0/agg.clicks as age_group_contrib_pct
FROM clicks c
     INNER JOIN lookup.user_age_group u ON c.user_id = u.k
     INNER JOIN
     (
         SELECT TIME_FLOOR(cc.__time, 'P1D') as _date,
                uu.v as age_group,
                COUNT(1) as clicks
           FROM clicks cc
           INNER JOIN lookup.user_age_group uu ON cc.user_id=uu.k
           GROUP BY 1, 2
     ) agg
        ON agg._date = TIME_FLOOR(c.__time, 'P1D') AND agg.age_group=u.v 
WHERE c.__time > CURRENT_TIMESTAMP  - INTERVAL '1' DAY
GROUP BY 1, 2, 3, agg.clicks 
ORDER BY 1, 5 DESC
LIMIT 10
'''
result = measure_query(sql, 20)
# saving all results for comparison 
join_results.append({"query_type":"subquery_join_lookup","results":result})
print_timestamp()
print(result)

### Query Pre-Joined Data
Avoiding the join altogether is usually the best option.
In the simply query case there are no joins because we processed the join and calculated the user's age_group at the time of ingestion. This saves a lot of processing at query time. You should always consider this option with Druid because while it will increase the size of the data (see below for different storage footprints of each method), it will save a lot of processing CPU and memory at query time.
Here are the same 2 queries but re-written using the `clicks_enhanced` table which we joined at ingestion above:

In [ ]:
sql='''
SELECT TIME_FLOOR(c.__time, 'P1D') as _date, 
  c.age_group, 
  count(distinct c.client_ip) as distint_ips
FROM clicks_enhanced c 
WHERE c.__time > CURRENT_TIMESTAMP  - INTERVAL '1' DAY
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 10
'''

result = measure_query(sql, 20)
# saving all results for comparison 
join_results.append({"query_type":"simple_prejoin","results":result})
print_timestamp()
print(result)

...even with the subquery to calculate totals by age group by day, it still avoids two joins by using the pre-joined datasource:

In [ ]:
sql = '''
SELECT TIME_FLOOR(c.__time, 'P1D') as _date, c.user_id,
  c.age_group, 
  count(1) as user_clicks,
  count(1)*100.0/agg.clicks as age_group_contrib_pct
FROM clicks_enhanced c
     INNER JOIN
     (
         SELECT TIME_FLOOR(cc.__time, 'P1D') as _date,
                cc.age_group,
                COUNT(1) as clicks
           FROM clicks_enhanced cc
           GROUP BY 1, 2
     ) agg
        ON agg._date = TIME_FLOOR(c.__time, 'P1D') AND agg.age_group=c.age_group 
WHERE c.__time > CURRENT_TIMESTAMP  - INTERVAL '1' DAY
GROUP BY 1, 2, 3, agg.clicks 
ORDER BY 1, 5 DESC
LIMIT 10
'''
result = measure_query(sql, 20)
# saving all results for comparison 
join_results.append({"query_type":"subquery_prejoin","results":result})
print_timestamp()
print(result)

### Results by Join Strategy

In [ ]:
# review the results
sorted_list = sorted(join_results, key=lambda x: x['query_type'])
sorted_list

In these results you can see the progression of response times. 
- The use of lookups improves performance over joining to another datasource, but it requires enough Heap to fit all lookups on all historical and ingestion tasks (Peon) JVMs.
- The pre-joined table performs even better and has the added advantage of not requiring any additional Heap.

If the data being joined changes over time and you wish to query with the latest values, consider a lookup. If the data does not change or the change is not desirable, then pre-join the data. The example of `age_group` as the joined property works well in the pre-join strategy because the user's age_group is important at the time they clicked, not when looking at old click rows based on their current age_group.

## Asynchronous Queries on MSQ Engine

The MSQ (Multi-stage Query Engine) engine works by shuffling rows between stages. All stages execute in tasks in parallel as much as they can.
![](assets/msq-engine.png)

Typically, as you did in the ingestion above, the MSQ engine is used to ingest external data, but it can also be used to query Druid datasources directly. It has the advantage of being able to do either `broadcast` or `sortMerge` joins. 



### Broadcast joins in MSQ
Broadcast joins execute similarly to the gather/scatter approach in that they send the whole right-hand dataset to all workers, but MSQ runs asynchronously and does not have the `maxSubQueryRows` limit. It will broadcast and load the whole right hand side table on all query_worker tasks.
![](assets/msq-broadcast.png)

In the first stage, `users` segments are scanned and all rows are broadcast to all workers. `clicks` data does not need to be distributed because all `users` rows are now available in all workers so join processing can proceed.

In [ ]:
def measure_async_query( sql: str, iterations: int, joinAlgorithm = 'broadcast', workers=3 ):
    req = sql_client.sql_request(sql)
    req.add_context("populateCache", "false")  # run without cacheing results to get a real sense of performance
    req.add_context("useCache", "false")  # do not use cached results
    req.add_context("sqlJoinAlgorithm", joinAlgorithm)
    req.add_context("maxNumTasks", workers)
    stats = []
    while (iterations>0):
      start = datetime.now()
      sql_client.async_sql(req)
      end = datetime.now()
      stats.append( (end - start).total_seconds() * 10**3 ) # add run time in milliseconds
      iterations -=1
    return f"Results = avg:{mean(stats)} ms   min:{min(stats)} ms  max:{max(stats)} ms"

In [ ]:
#MSQ async query using broadcast join
sql='''
SELECT TIME_FLOOR(c.__time, 'P1D') as _date, 
  round( TIMESTAMPDIFF(YEAR, TIME_PARSE(u.dob), CURRENT_TIMESTAMP),-1) as age_group, 
  count(distinct c.client_ip) as distint_ips
FROM clicks c
     INNER JOIN users u
        ON c.user_id=u.user_id
WHERE c.__time > CURRENT_TIMESTAMP  - INTERVAL '1' DAY
GROUP BY 1, round( TIMESTAMPDIFF(YEAR, TIME_PARSE(u.dob), CURRENT_TIMESTAMP),-1) 
ORDER BY 3 DESC
LIMIT 10
'''
result = sql_client.async_sql(sql)
result=pandas.json_normalize(result.rows)
print_timestamp()
result

In [ ]:
print(measure_async_query(sql,10)) # using default broadcast join
print_timestamp()

MSQ has an inherent disadvantage over Scatter/Gather when it comes to response time for many queries. It must first initialize the `query_controller` and `query_worker` tasks and only then begin to process the query.  You must also make sure that you have enough worker slots available for them. If any of the tasks are waiting to be assigned, the whole query will wait before it starts executing stages. It will fail after a timeout if worker slots do not become available.
MSQ reads the segment files directly from Deep Storage which is also typically slower than the local storage that Historicals use and there is no caching of segment files. It has the advantage that it can access data that is not cached in the historicals but still exists in Deep Storage. So it is best for longer running queries on broader or older timeframes.

As queries are running you can see the `query_controller` and `query_worker` tasks being spawned for each query in the [tasks view of the Druid Console](http://localhost:8888/unified-console.html#tasks).


### Sort Merge joins in MSQ
SortMerge joins execute by hash distributing both sides of the join among all the `query_workers` using the join columns to calculate the hash. The end result is that the rows from each of the tables that correspond to a given join column value will end up on the same worker:
![](assets/msq-sortmerge.png)

In [ ]:
#MSQ async query using sortMerge
print(measure_async_query(sql,10,joinAlgorithm='sortMerge'))
print_timestamp()

In this case, sortMerge joins did not help which makes sense. A broadcast of 4000 `users` rows to the workers is much faster than having to redistribute both `clicks` and `users`. But if the two tables are large, a broadcast of one of them will not be possible because it would not fit in the Heap of the `query_workers`.
Try the following query that joins `clicks` with `clicks` in using `broadcast` and `sortMerge` algorithms to see this effect.

In [ ]:
sql = '''
SELECT c.__time, c.user_id, c.client_ip, MAX(c2.__time) prior_time_in_session, count(*) prior_events_in_session 
FROM clicks c
     INNER JOIN 
      clicks c2 ON c.user_id=c2.user_id AND c.client_ip = c2.client_ip
WHERE c.__time > (CURRENT_TIMESTAMP  - INTERVAL '1' DAY) AND
      c2.__time > (CURRENT_TIMESTAMP - INTERVAL '2' DAY) AND
      c2.__time < c.__time
GROUP BY 1,2,3
LIMIT 10
'''
try:
    result = sql_client.async_sql(sql)
    result.rows
except Exception as ex:
    print(f"Error: {ex}")

As expected, the query fails because it cannot broadcast that much data to the workers. As the suggestion in the error indicates, the next run of the same query uses `sortMerge` for the join algorithm which does not have such limits. 

It takes a couple of minutes to complete, while it is running you can see its progress by attaching to it in the [Druid Console](http://localhost:8888/unified-console.html#workbench):

![](assets/msq-attach.png)

In [ ]:
req = sql_client.sql_request(sql)
req.add_context("sqlJoinAlgorithm", 'sortMerge')
req.add_context("maxNumTasks", 3)
result = sql_client.async_sql(req)
result=pandas.json_normalize(result.rows)
print_timestamp()
result

## UNION ALL Queries
In some scenarios, UNION ALL queries can be used to bring data from different tables together instead of doing a join.
There are two scenarios where [UNION ALL is supported](https://druid.apache.org/docs/latest/querying/sql#union-all).
- Broker UNION ALL brings together the results of two subqueries. Each subquery is processed in its own scatter/gather process, the results of each subquery are returned to the Broker and the Broker returns the merged results to the client.
- Historical/Peon UNION ALL, takes segments from two different tables and applies the same processing in all Data Servers including Historicals and Streaming Ingestion tasks. The Broker finds the relevant segments from the UNION ALL tables and submits the same query request to the data servers that own them, essentially treating multiple tables as if they were a single table. This means that the same column names must be selected from both of the tables, which can be limiting but in the rest of the notebook we'll show how it can be a powerful tool for querying multiple datasources together and how they can effectively execute joins under certain conditions.


### Generate `sales` and `sales_forecast` Data
In the retail industry, it is common to have a forecast of how many units you will sell in some period of time. Sales data is then aggregated to the same level as the forecast to measure progress against it. The forecast data is necessarily generated before the sales data, it wouldn't be a forecast otherwise. This creates a situation where pre-joining the data is not viable and the datasets can both be large. In this example the forecast data isn't very big but it will help illustrate how UNION ALL can be used to resolve this type of query.

The next cell generates detailed `sales data` from clickstream `purchase` events and corresponding `forecast data` per country per product per day. We'll then ingest it in slightly different ways to show different UNION ALL strategies. The forecast values are somewhat random and unrealistic, so don't try to make sense of them. Instead focus on the ingestion design and query pattern.

In [ ]:
# simulate forecast for last 2 weeks 
gen_hours=24*14  
gen_now = datetime.now().replace(hour=0,minute=0,second=0) - timedelta(hours=gen_hours)
gen_start_time = gen_now.strftime("%Y-%m-%d %H:%M:%S")

headers = {
  'Content-Type': 'application/json'
}

datagen_request = {
    "name": "forecast",
    "target": { "type": "file", "path":"forecast.json"  },
    "config_file": "clickstream/sales_forecast.json", 
    "time": f"{gen_hours}h",
    "concurrency":1040,
    "time_type":gen_start_time
}
datagen.post("/start", json.dumps(datagen_request), headers=headers)
wait_for_datagen("forecast")
print_timestamp("after forecast data generation")

datagen_request = {
    "name": "clicks",
    "target": { "type": "file", "path":"clicks.json"  },
    "config_file": "clickstream/clickstream.json", 
    "time": f"{gen_hours}h",
    "concurrency":20,
    "time_type":gen_start_time
}
datagen.post("/start", json.dumps(datagen_request), headers=headers)
wait_for_datagen("clicks")
print_timestamp("after sales data generation")

### Natural Ingestion
For the first two query patterns, we ingest the data mostly "as is". 

Run the following cell to:
- ingest "sales" from clickstream data filtering for only "event_type"=`purchase` and adds a "quantity" constant set to 1 so we can aggregate sales quantity.
- ingest "sales_forecast" by adding an "event_type" of `forecast` and renaming the source "forecast_quantity" to "quantity".

The "event_type" is used to distinguish between `purchase` and `forecast` values in one UNION ALL scenario below. The same name for the "quantity" column enables the UNION ALL technique where all the columns selected from the unioned tables must be exactly the same.

In [ ]:
# ingest sales from click data by 
# - filtering source for purchase events with "event_type"='purchase'
# - adding a column for "quantity" with a constant value of 1
sql='''
REPLACE INTO "sales" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["http://datagen:9999/file/clicks.json"]}',
    '{"type":"json"}'
  )
) EXTEND ("time" VARCHAR, "user_id" VARCHAR, "event_type" VARCHAR, "client_ip" VARCHAR, "client_device" VARCHAR, "client_lang" VARCHAR, "client_country" VARCHAR, "referrer" VARCHAR, "keyword" VARCHAR, "product" VARCHAR))
SELECT
  TIME_PARSE("time") AS "__time",
  "user_id",
  "event_type",
  "client_country",
  "product",
  1 as "quantity"
FROM "ext"
WHERE "event_type"='purchase'  
PARTITIONED BY DAY
'''
display.run_task(sql)
print_timestamp()

# ingest forecast data with a couple of adjustments
# - add column expression 'forecast' as event_type, such that we can distinguish between purchases and forecasts when using UNION ALL
# - change the name of the "forecast_quantity" column to "quantity" so that columns names match when using UNION ALL
sql='''
REPLACE INTO "sales_forecast" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["http://datagen:9999/file/forecast.json"]}',
    '{"type":"json"}'
  )
) EXTEND ("time" VARCHAR, "client_country" VARCHAR, "product" VARCHAR, "forecast_quantity" INTEGER))
SELECT
  TIME_PARSE("time") AS "__time",
  'forecast' as "event_type",
  "client_country",
  "product",
  "forecast_quantity" as "quantity"
FROM "ext"
PARTITIONED BY DAY
'''
display.run_task(sql)
print_timestamp()


# make sure both tables are available before moving on
sql_client.wait_until_ready('sales')
sql_client.wait_until_ready('sales_forecast')
print_timestamp()

### Independent Queries - App Joins Results

In order to compare forecast to sales, we will need to aggregate both at the same time granularity and with the same dimensions. 

One way to resolve this, is to issue two different queries, one provides the appropriate forecast aggregation by week and the other the corresponding aggregation of sales also by week. 

In the interest of keeping the results short, we filter for just 3 products and one country. 

The join of the two resultsets is done here using pandas dataframes which run on the jupyter kernel, so this is the application resolving the join:

In [ ]:
# we can read data independently from each source, and join it in python
start_at = print_timestamp()
sql = '''
    SELECT 
       TIME_FLOOR(__time, 'P1W') week_start, 
       "client_country",
       "product",
       SUM("quantity") "sales_quantity"
    FROM "sales"
    WHERE __time > TIME_FLOOR( CURRENT_TIMESTAMP, 'P1D') - INTERVAL '7' DAY
    AND "product" in ('Slinky', 'Magic 8-ball', 'Pet rock')
    AND "client_country" = 'Japan'
    GROUP BY 1,2,3
'''
sales_data = pandas.json_normalize(sql_client.sql(sql)).set_index(['week_start','client_country','product'])

sql = '''
    SELECT 
       TIME_FLOOR(__time, 'P1W') week_start, 
       "client_country",
       "product",
       SUM("quantity") "forecast_quantity"
    FROM "sales_forecast"
    WHERE __time > TIME_FLOOR( CURRENT_TIMESTAMP, 'P1D') - INTERVAL '7' DAY
    AND "product" in ('Slinky', 'Magic 8-ball', 'Pet rock')
    AND "client_country" = 'Japan'
    GROUP BY 1,2,3
'''
sales_forecast_data = pandas.json_normalize(sql_client.sql(sql)).set_index(['week_start','client_country','product'])

# we join it here on the "application" side:
result = sales_data.join(sales_forecast_data, on=['week_start','client_country','product'])
end_at=print_timestamp()
print(f"Duration: {(end_at-start_at).total_seconds()*1000} ms ")
result

### Single Query with UNION ALL - Sales and Forecast in Different Result Rows
Given that both "sales" and "sales_forecast" tables have the same set of columns for the query, you can use a UNION ALL to process both aggregations in a single pass and return the results sorted such that the corresponding sales "quantity" and forecast "quantity" are in consecutive for each country, product combination.

Notice that the UNION ALL selects the same columns from both tables. This is a requirement for this query pattern. If you use different projections for each table, you will get an error.

The outer query with specifies the aggregation which will be executed in parallel on all segments selected from both tables in the UNION ALL. This includes the filters which will be used by the Broker for pruning which segments of each table are relevant as well as filter processing in the Historicals using column dictionaries and indexes. Since the columns are the same, the same set of historical processing instructions can be applied to segments of either table.

Try it out:

In [ ]:
sql = '''
    SELECT 
       TIME_FLOOR(__time, 'P1W') week_start, 
       "client_country",
       "product",
       "event_type",
       SUM("quantity") "quantity"
    FROM 
    (
        SELECT __time, "client_country", "product", "event_type", "quantity" FROM "sales"
    UNION ALL
        SELECT __time, "client_country", "product", "event_type", "quantity" FROM "sales_forecast"
    )
    WHERE  __time > TIME_FLOOR( CURRENT_TIMESTAMP, 'P1D') - INTERVAL '7' DAY
    AND "product" in ('Slinky', 'Magic 8-ball', 'Pet rock')
    AND "client_country" = 'Japan'
    GROUP BY 1,2,3,4
    ORDER BY 1,2,3,4
'''

result=pandas.json_normalize(sql_client.sql(sql))
print_timestamp()
result

Your specific results will vary because the data generator is somewhat random. In my results I can see that some country,product combinations do not have a forecast, so there is only a purchase row. An application processing these results would need to keep that in mind. The same would be true in sales if, for a particular period, there are no sales of a given product/country combo.

In [ ]:
# measure multiple query runs
print(measure_query(sql, 20))
print_timestamp()

### Single Query UNION ALL - Sales and Forecast "joined" through Aggregation 
Another form of using the UNION ALL can produce the fully joined results, but it will require that the data be ingested with some changes:
- add a NULL valued column called "forecast_quantity" to the sales table
- add a NULL valued column called "quantity" to the forecast table

This allows you to select both the sales "quantity" and "forecast_quantity" in the unioned tables such that resulting aggregate rows contain both. The end result is effectively a join of the two data sets through aggregated to the same granularity.


In [ ]:
# ingest sales from click data, adding a NULL valued column called "forecast_quantity"
sql='''
REPLACE INTO "sales_u" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["http://datagen:9999/file/clicks.json"]}',
    '{"type":"json"}'
  )
) EXTEND ("time" VARCHAR, "user_id" VARCHAR, "event_type" VARCHAR, "client_ip" VARCHAR, "client_device" VARCHAR, "client_lang" VARCHAR, "client_country" VARCHAR, "referrer" VARCHAR, "keyword" VARCHAR, "product" VARCHAR))
SELECT
  TIME_PARSE("time") AS "__time",
  "user_id",
  "event_type",
  "client_country",
  "product",
  1 as "quantity",
  NULL as "forecast_quantity"
FROM "ext"
WHERE "event_type"='purchase'  
PARTITIONED BY DAY
'''
display.run_task(sql)
print_timestamp()


# ingest sales forecast data, adding a NULL valued column called "quantity"
sql='''
REPLACE INTO "sales_forecast_u" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["http://datagen:9999/file/forecast.json"]}',
    '{"type":"json"}'
  )
) EXTEND ("time" VARCHAR, "client_country" VARCHAR, "product" VARCHAR, "forecast_quantity" INTEGER))
SELECT
  TIME_PARSE("time") AS "__time",
  "client_country",
  "product",
  NULL as "quantity",
  "forecast_quantity"
FROM "ext"
PARTITIONED BY DAY
'''
display.run_task(sql)
print_timestamp()


# make sure both tables are available before moving on
sql_client.wait_until_ready('sales_u')
sql_client.wait_until_ready('sales_forecast_u')
print_timestamp()


This query pattern works by calculating aggregation in parallel for "sales" segments and "sales_forecast" segments in the data servers. After initial segment level aggregation, the results from "sales" segments will have a NULL SUM for "forecast_quantity" and results for "sales_forecast" segments will have NULL on the SUM of "quantity" column. As these results are merged, aggregations of NULL and non-NULL values for each of the columns will result in a single row for each week,country,product combo that has both the "quantity" and "forecast_quantity". 

Even if a given product/country does not have a forecast or doesn't have sales in a particular period, the resulting row will still appear as long as it exists in either of the tables.

Try it out:

In [ ]:
sql = '''
    SELECT 
       TIME_FLOOR(__time, 'P1W') week_start, 
       "client_country",
       "product",
       SUM("quantity") as "sales_quantity",
       SUM("forecast_quantity") as "forecast_quantity"
    FROM 
    (
        SELECT __time, "client_country", "product", "quantity", "forecast_quantity" FROM "sales_u"
    UNION ALL
        SELECT __time, "client_country", "product", "quantity", "forecast_quantity" FROM "sales_forecast_u"
    )
    WHERE __time > TIME_FLOOR( CURRENT_TIMESTAMP, 'P1D') - INTERVAL '7' DAY
    AND "product" in ('Slinky', 'Magic 8-ball', 'Pet rock')
    AND "client_country" = 'Japan'
    GROUP BY 1,2,3
    ORDER BY 1,2,3
'''

result=pandas.json_normalize(sql_client.sql(sql))
print_timestamp()
result

In [ ]:
# measure multiple query runs
print(measure_query(sql, 100))
print_timestamp()

## Clean up

Run the following cell to remove the tables created throughout this notebook.

In [ ]:
druid.datasources.drop("clicks")
druid.datasources.drop("users")
druid.datasources.drop("clicks_enhanced")
druid.datasources.drop("sales")
druid.datasources.drop("sales_forecast")
druid.datasources.drop("sales_u")
druid.datasources.drop("sales_forecast_u")

## Summary

Druid can process data join operations at query time in different ways:
<br>
- In the scatter/gather model:
    * joined tables are first scanned and the result is broadcast to all data servers to process the join with the first table
    * joining a table to subqueries means the subqueries are processed as separate queries, and their results are then broadcast to join with the first table
    * order of the joined tables is important, the larger table should be listed first such that it drives parallelization of the join
    * there is a limit to how much data can be broadcast set by either `maxSubQueryRows` or `maxSubQueryBytes` limits
    * you can raise the subquery limits but it comes at the cost of more Heap memory used on the Broker, Historicals, and Streaming Tasks to resolve the query and more data transmitted from the Broker to all the others involved in the query
<br>
<br>
- In the MSQ engine query model:
    * meant for longer running queries with more complex joins
    * queries are generally slower because:
        * they need to spawn workers in order to run
        * they read the segment data from Deep Storage
    * joins can use either broadcast or sortMerge join algorithms
    * with sortMerge joins, large fact to fact joins are possible 
<br>
<br>
- UNION ALL can be an powerful alternative to large joins
    * for correctness, queries must aggregate to the coarsest time granularity of the unioned tables
    * all columns involved in the query need to exist on all unioned tables
    * by adding NULL valued columns with the same name to the other table(s) UNION ALL aggregation can effectively do fact to fact joins    